<a href="https://colab.research.google.com/github/Harikrishnang37/Content_Reccomendation_Engine/blob/main/ReccomendationEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Reccomendation Engine

In this project, we are going to use the data from a movie database to reccomend content to a user based on their previous ratings.

In [ ]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')

!unzip -o -j moviedataset.zip 

--2022-06-09 19:18:40--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  18.4MB/s    in 7.2s    

2022-06-09 19:18:49 (21.3 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv        

In [ ]:


#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
#movies_df.head()


#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
#movies_df.head()

#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
#movies_df.head()


#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, 
                                     genre] = int(1)

       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)

In [ ]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
            
         ] 
inputMovies = pd.DataFrame(userInput)
print(inputMovies)

                 title  rating
0  Breakfast Club, The     5.0
1            Toy Story     3.5
2              Jumanji     2.0
3         Pulp Fiction     5.0
4                Akira     4.5


In [ ]:
#Filtering out the movies by title
# no iteration done it matches the title in one shot

inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)

#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.

#print(inputMovies)
#print(inputId)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]


In [ ]:
userMovies.drop(["movieId", "title", "genres", "year"], inplace = True, axis = 1)




/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
ratingArray = np.array(inputMovies['rating'])
ratingArray = ratingArray.reshape(1,len(inputMovies['rating']))

genreArray = np.array(userMovies)

scores = np.dot(ratingArray, genreArray)


In [ ]:
#Normalizing:

scoresArray = np.array(scores[0])

scoresSum =0
for i in scores[0]:
  scoresSum += i

scoresArray = scoresArray / scoresSum


In [ ]:
#Now, let us take the entire dataset of movies and genres:

fullListOfMovies = moviesWithGenres_df.drop(['genres','year'],inplace = False, axis =1)


In [ ]:
#Now, let us replace the '1's in this dataset with the corresponding normalized scores.

fullListOfMovies['Adventure'] = fullListOfMovies['Adventure'] * scoresArray[0]
fullListOfMovies['Animation'] = fullListOfMovies['Animation'] * scoresArray[1]
fullListOfMovies['Children'] = fullListOfMovies['Children'] * scoresArray[2]
fullListOfMovies['Comedy'] = fullListOfMovies['Comedy'] * scoresArray[3]
fullListOfMovies['Fantasy'] = fullListOfMovies['Fantasy'] * scoresArray[4]
fullListOfMovies['Romance'] = fullListOfMovies['Romance'] * scoresArray[5]
fullListOfMovies['Drama'] = fullListOfMovies['Drama'] * scoresArray[6]
fullListOfMovies['Action'] = fullListOfMovies['Action'] * scoresArray[7]
fullListOfMovies['Crime'] = fullListOfMovies['Crime'] * scoresArray[8]
fullListOfMovies['Thriller'] = fullListOfMovies['Thriller'] * scoresArray[9]
fullListOfMovies['Horror'] = fullListOfMovies['Horror'] * scoresArray[10]
fullListOfMovies['Mystery'] = fullListOfMovies['Mystery'] * scoresArray[11]
fullListOfMovies['Sci-Fi'] = fullListOfMovies['Sci-Fi'] * scoresArray[12]
fullListOfMovies['IMAX'] = fullListOfMovies['IMAX'] * scoresArray[13]
fullListOfMovies['Documentary'] = fullListOfMovies['Documentary'] * scoresArray[14]
fullListOfMovies['War'] = fullListOfMovies['War'] * scoresArray[15]
fullListOfMovies['Musical'] = fullListOfMovies['Musical'] * scoresArray[16]
fullListOfMovies['Western'] = fullListOfMovies['Western'] * scoresArray[17]
fullListOfMovies['Film-Noir'] = fullListOfMovies['Film-Noir'] * scoresArray[18]
fullListOfMovies['(no genres listed)'] = fullListOfMovies['(no genres listed)'] * scoresArray[19]



In [ ]:
fullListOfMovies['Final Score'] = fullListOfMovies.drop(['movieId'],1).sum(axis = 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [ ]:
sortedMovieList = fullListOfMovies.sort_values(by = 'Final Score', ascending = False)

sortedMovieList.head()

,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed),Final Score
4923,5018,Motorama,0.13986,0.000000,0.000000,0.188811,0.076923,0.0,0.13986,0.000000,...,0.0,0.062937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.748252
8605,26093,"Wonderful World of the Brothers Grimm, The",0.13986,0.111888,0.076923,0.188811,0.076923,0.0,0.13986,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.734266
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,0.13986,0.111888,0.000000,0.188811,0.076923,0.0,0.13986,0.062937,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.720280
33509,148775,Wizards of Waverly Place: The Movie,0.13986,0.000000,0.076923,0.188811,0.076923,0.0,0.13986,0.000000,...,0.0,0.062937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.685315
25218,117646,Dragonheart 2: A New Beginning,0.13986,0.000000,0.000000,0.188811,0.076923,0.0,0.13986,0.062937,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.678322
